In [ ]:
import sys
from pathlib import Path
import shutil
import json
import csv
import re
import math

from shapely.geometry import Point
import requests

ROOT = Path('..')
sys.path.append(str(ROOT))

import wavetrace as wt

%load_ext autoreload
%autoreload 2

TMP_DIR = ROOT/'TMP_DIR'


In [ ]:
# # Visually check that SRTM tiles cover NZ by 
# # making polygons and viewing them at https://geojson.io.

# tids = wt.SRTM_NZ_TILE_IDS
# collection = {
#     'type': 'FeatureCollection', 
#     'features': list(map(wt.build_feature, tids))
#     }
# json.dumps(collection)

In [ ]:
# Initialize

wt.rm_paths(TMP_DIR)
#transmitters_path = ROOT/'tests'/'data'/'transmitters.csv'
transmitters_path = ROOT/'tests'/'data'/'transmitters_single.csv'
high_definition = False
if high_definition:
    topography_path = TMP_DIR/'srtm1'
else:
    topography_path = TMP_DIR/'srtm3'


In [ ]:
# Process transmitters

out_path = TMP_DIR/'splat_files'   
wt.process_transmitters(transmitters_path, out_path)

% ls {out_path}

In [ ]:
# Download topography

transmitters = wt.read_transmitters(transmitters_path)
print('transmitters=', transmitters)

tids = wt.get_covering_tiles_ids(transmitters)
print('Tiles to download =', tids)

wt.download_topography(tids, topography_path, 
  high_definition=high_definition)

% ls {topography_path}

In [ ]:
# Process topography

out_path = TMP_DIR/'splat_files'
wt.process_topography(topography_path, out_path, 
  high_definition=high_definition)

% ll -h {out_path}

In [ ]:
# Compute coverage

in_path = TMP_DIR/'splat_files'
out_path = TMP_DIR/'coverage_reports'
wt.compute_coverage(in_path, out_path, high_definition=high_definition, make_shp=True)

% ls {out_path}

In [ ]:
# Compute satellite line-of-sight for the Optus D2 geostationary satellite 
# at 152 degrees longitude

in_path = (TMP_DIR/'srtm3'/'S36E173.SRTMGL3.hgt.zip').resolve()
satellite_lon = 152
tile_id = wt.get_tile_id(in_path)
out_path = TMP_DIR/'{!s}_satellite_los.tif'.format(tile_id)
wt.compute_satellite_los(in_path, satellite_lon, out_path)

In [ ]:
# Clean up

wt.rm_paths(TMP_DIR)